In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Datos limpios

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 24.8 s, sys: 902 ms, total: 25.7 s
Wall time: 25.7 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Libros viajeros

In [7]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}

In [8]:
sucursales_bogota_no_blaa = ['CASAGO', 'IEC', 'ESPE', 'CATMAN']

In [9]:
sucursales_bogota = sucursales_bogota_no_blaa + ['BLAA']

## Modelos de viaje

In [10]:
# MODEL   SUC_LIBRO  SUC_PRéST  SUC_DEVOLUCIóN    %
#  mABC      A          B          C             0.23%
#  mBBC      B          B          C             2.05%
#  mABB      A          B          B            13.61%
#  mABA      A          B          A             2.20%
#  mBBB      B          B          B            81.91%

#### Modelo ABC

In [11]:
prestamos_mABC_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [12]:
prestamos_mABC_df[['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION']].head()

,SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION
957,ARM,CASAGO,BLAA
1724,ARM,CASAGO,BLAA
2554,ARM,CASAGO,BLAA
2946,ARM,CASAGO,BLAA
3789,ARM,CASAGO,BLAA


In [13]:
len(prestamos_mABC_df)

7063

In [14]:
per = 100 * len(prestamos_mABC_df) / len(prestamos_df)
'%.2f%%' % per

'0.23%'

#### Modelo BBC

In [33]:
prestamos_mBBC_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [34]:
prestamos_mBBC_df[['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION']].head()

,SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION
308,ARM,ARM,PER
1280,ARM,ARM,BLAA
1564,ARM,ARM,PER
1808,ARM,ARM,PER
1905,ARM,ARM,BLAA


In [35]:
len(prestamos_mBBC_df)

62598

In [36]:
per = 100 * len(prestamos_mBBC_df) / len(prestamos_df)
'%.2f%%' % per

'2.05%'

#### Modelo ABB

In [19]:
prestamos_mABB_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [20]:
prestamos_mABB_df[['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION']].head()

,SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION
20,ARM,BUC,BUC
67,ARM,VIL,VIL
102,ARM,CASAGO,CASAGO
118,ARM,PER,PER
141,ARM,PER,PER


In [21]:
len(prestamos_mABB_df)

415679

In [22]:
per = 100 * len(prestamos_mABB_df) / len(prestamos_df)
'%.2f%%' % per

'13.61%'

#### Modelo ABA

In [23]:
prestamos_mABA_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [24]:
prestamos_mABA_df[['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION']].head()

,SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION
4672,BLAA,CATMAN,BLAA
4752,BLAA,CATMAN,BLAA
4788,BLAA,ESPE,BLAA
4798,BLAA,CASAGO,BLAA
4857,BLAA,ESPE,BLAA


In [25]:
len(prestamos_mABA_df)

67289

In [26]:
per = 100 * len(prestamos_mABA_df) / len(prestamos_df)
'%.2f%%' % per

'2.20%'

#### Modelo 5

In [27]:
prestamos_mBBB_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [28]:
prestamos_mBBB_df[['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION']].head()

,SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION
0,ARM,ARM,ARM
1,ARM,ARM,ARM
2,ARM,ARM,ARM
3,ARM,ARM,ARM
4,ARM,ARM,ARM


In [29]:
len(prestamos_mBBB_df)

2501731

In [30]:
per = 100 * len(prestamos_mBBB_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

#### Total

In [37]:
len(prestamos_mABC_df) + len(prestamos_mBBC_df) + len(prestamos_mABB_df) + len(prestamos_mABA_df) + len(prestamos_mBBB_df)

3054360

In [38]:
len(prestamos_df)

3054360

#### % préstamos sin viaje

In [39]:
per = 100 * len(prestamos_mBBB_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

#### % préstamos con viaje

In [40]:
per = 100 * (len(prestamos_mABC_df) + len(prestamos_mBBC_df) + len(prestamos_mABB_df) + len(prestamos_mABA_df)) / len(prestamos_df)
'%.2f%%' % per

'18.09%'

### Modelo ABC

In [41]:
%%time

gABC = nx.MultiDiGraph()

for p in prestamos_mABC_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not gABC.has_node(s_libro):
        gABC.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    gABC.node[s_libro]['weight'] += 1
    if not gABC.has_node(s_prest):
        gABC.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    gABC.node[s_prest]['weight'] += 1
    if not gABC.has_node(s_devol):
        gABC.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    gABC.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not gABC.has_edge(s_libro, s_prest, key):
            gABC.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        gABC[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not gABC.has_edge(s_prest, s_devol, key):
            gABC.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        gABC[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 1.74 s, sys: 0 ns, total: 1.74 s
Wall time: 1.74 s


In [42]:
gABC.number_of_nodes()

32

In [43]:
gABC.number_of_edges()

680

In [44]:
for s in sucursales_bogota_no_blaa:
    if s in gABC:
        gABC.remove_node(s)

In [45]:
gABC.number_of_nodes()

29

In [46]:
gABC.number_of_edges()

535

In [47]:
nx.write_gexf(gABC, 'modeloABC.gexf')

### Modelo BBC

In [48]:
%%time

gBBC = nx.MultiDiGraph()

for p in prestamos_mBBC_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not gBBC.has_node(s_libro):
        gBBC.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    gBBC.node[s_libro]['weight'] += 1
    if not gBBC.has_node(s_prest):
        gBBC.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    gBBC.node[s_prest]['weight'] += 1
    if not gBBC.has_node(s_devol):
        gBBC.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    gBBC.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not gBBC.has_edge(s_libro, s_prest, key):
            gBBC.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        gBBC[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not gBBC.has_edge(s_prest, s_devol, key):
            gBBC.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        gBBC[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 15.2 s, sys: 70.9 ms, total: 15.3 s
Wall time: 15.3 s


In [49]:
gBBC.number_of_nodes()

33

In [50]:
gBBC.number_of_edges()

401

In [51]:
for s in sucursales_bogota_no_blaa:
    if s in gBBC:
        gBBC.remove_node(s)

In [52]:
gBBC.number_of_nodes()

29

In [53]:
gBBC.number_of_edges()

369

In [54]:
nx.write_gexf(gBBC, 'modeloBBC.gexf')

### Modelo ABB

In [56]:
%%time

gABB = nx.MultiDiGraph()

for p in prestamos_mABB_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not gABB.has_node(s_libro):
        gABB.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    gABB.node[s_libro]['weight'] += 1
    if not gABB.has_node(s_prest):
        gABB.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    gABB.node[s_prest]['weight'] += 1
    if not gABB.has_node(s_devol):
        gABB.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    gABB.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not gABB.has_edge(s_libro, s_prest, key):
            gABB.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        gABB[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not gABB.has_edge(s_prest, s_devol, key):
            gABB.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        gABB[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 1min 31s, sys: 162 ms, total: 1min 31s
Wall time: 1min 31s


In [57]:
gABB.number_of_nodes()

33

In [58]:
gABB.number_of_edges()

806

In [59]:
for s in sucursales_bogota_no_blaa:
    if s in gABB:
        gABB.remove_node(s)

In [60]:
gABB.number_of_nodes()

29

In [61]:
gABB.number_of_edges()

705

In [62]:
nx.write_gexf(gABB, 'modeloABB.gexf')

### Modelo ABA

In [63]:
%%time

gABA = nx.MultiDiGraph()

for p in prestamos_mABA_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not gABA.has_node(s_libro):
        gABA.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    gABA.node[s_libro]['weight'] += 1
    if not gABA.has_node(s_prest):
        gABA.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    gABA.node[s_prest]['weight'] += 1
    if not gABA.has_node(s_devol):
        gABA.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    gABA.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not gABA.has_edge(s_libro, s_prest, key):
            gABA.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        gABA[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not gABA.has_edge(s_prest, s_devol, key):
            gABA.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        gABA[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 14.6 s, sys: 27.7 ms, total: 14.6 s
Wall time: 14.7 s


In [64]:
gABA.number_of_nodes()

32

In [65]:
gABA.number_of_edges()

378

In [66]:
for s in sucursales_bogota_no_blaa:
    if s in gABA:
        gABA.remove_node(s)

In [67]:
gABA.number_of_nodes()

29

In [68]:
gABA.number_of_edges()

332

In [69]:
nx.write_gexf(gABA, 'modeloABA.gexf')